# Assignment - 2: Data Representation and Point Cloud Operations

Team Name: R2D2 

Roll number: 2019101120, 2019111030

# Instructions

- Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment.
- Save all your results in ```results/<question_number>/<sub_topic_number>/```
- The **References** section provides you with important resources to solve the assignment.
- Make sure your code is modular since you may need to reuse parts for future assignments.
- Answer the descriptive questions in your own words with context & clarity. Do not copy answers from online resources or lecture notes.
- The **deadline** for this assignment is on 26/09/2021 at 11:55pm. Please note that there will be no extensions.
- Plagiarism is **strictly prohibited**.

# Submission Instructions

1. Make sure your code runs without any errors after reinitializing the kernel and removing all saved variables.
2. After completing your code and saving your results, zip the folder with name as ``Team_<team_name>_MR2021_Assignment_<assignment_number>.zip``

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import open3d as o3d
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Introduction to types of Transformations and Homogeneous coordinates

In robotics applications, it is inevitable to keep track of the frames of multiple objects/worlds. These frames can be transformations from one coordinate frame to the other. **Homogeneous coordinates** help in keeping track of various coordinate frames and allow performing composition of various transforms. We will first try to understand between types of transformations and their invariant properties.
1. What is the difference between Affine, Similarity, and Euclidean transform? What are the invariant properities of each type of transform?
2. Watch this [video](https://www.youtube.com/watch?v=PvEl63t-opM) to briefly understand homogeneous coordinates. What are points at infinity? What type of transformation can you apply to transform a point from infinity to a point that is not at infinity? 
3. Using homogeneous coordinates we can represent different types of transformation as point transforms vs. frame transforms. Concatenation of transforms (whether you post multiply transformation matrices or pre-multiply transformation matrices) depends on the problem and how you are viewing it. Try to understand the difference between frame vs. point transformations from this [video](https://youtu.be/Za7Sdegf8m8?t=1834). Let's assume that our camera and world frames are coinciding with each other. We need to estimate the camera to world **frame** transformation matrix after applying the transformations defined below in terms of $T_i$.We apply **frame** transform to move the camera in the world in the following order:
    1. $T_1$ from the camera coordinate frame.
    2. $T_2$ from the world coordinate frame.
    3. $T_3$ from the world coordinate frame.
    4. $T_4$ from the camera coordinate frame.
    5. $T_5$ from the camera coordinate frame.


1. Similarity and Euclidean transformations are special cases of affine transformations. An affine transformation is a combination of a non-singular linear transformation and translation, whereas similarity transformation is a combination of translation, rotation, uniform scaling , and reflection. A Euclidean transformation is a case of affine transformation where the linear transformation is rotation ($A \space A^{T} = A^{T}A = I$). The geometry of Euclidean symmetry is carried forward. The invariants for affine transformations are points, straight lines, planes, parallelism, ratios of lengths along parallel sides. The invariants for Euclidean transformations are lengths, angles and all the preserved properties of affine transformations. For similarity transformations, the angles, shape and trace of the matrix is invariant.

2. Points at infinity are points that are infinitively far away with finite coordinates. In the transformation from Euclidean to Homogenous coordinates, the third coordinate of the vector is made 0, so that when we are going back to Euclidean representation, we divide by 0 and the points are infinitely far. The direction to the point is still maintained, so even if the point is infinitively far away, we can still estimate the direction to that point. This property is useful while working with cameras and infinitely far points.

3. Since the transformation from camera frame is post-multiplied and transformation from world frame is pre-multiplied, the transformations are as follows:
    $ = T_{i} * T_{1}$
    
    $ = T_{2} * T_{i} * T_{1}$
    
    $ = T_{3} * T_{2} * T_{i} * T_{1}$
    
    $ = T_{3} * T_{2} * T_{i} * T_{1} * T_{4}$
    
    $ = T_{3} * T_{2} * T_{i} * T_{1} * T_{4} * T_{5}$

# Visualise the Data

Point clouds are a collection of points that represent a 3D shape or feature. Each point has its own set of X, Y and Z coordinates and in some cases additional attributes. A popular way to obtain this is by photogrammetry, though here we will use LiDAR data.

LiDAR is a remote sensing process which collects measurements used to create 3D models and maps of objects and environments. Using ultraviolet, visible, or near-infrared light, LiDAR gauges spatial relationships and shapes by measuring the time it takes for signals to bounce off objects and return to the scanner.

1. Download the data from [here](https://iiitaphyd-my.sharepoint.com/:f:/g/personal/venkata_surya_students_iiit_ac_in/EnYAMaTVIhJItzKYqtahE30BRKB6p6UfHN3TyJzvo6Mw0g?e=PegWds). It contains the LIDAR sensor output and odometry information per frame.

    The .bin files contain the 3D point cloud captured by the LIDAR in this format - x, y, z, and reflectance. 

    The odometry information is given in the `odometry.txt` file, which is a 12 element vector. Reshape each of the first 77 rows to a 3x4 matrix to obtain the pose.
    

2. Obtain the point cloud from this and visualise for 1-2 frames.

In [3]:
#Reshaping first 77 rows
ground_truth = np.loadtxt('data/odometry.txt')
poses = [0]*77
for i in range(77):
    poses[i] = np.reshape(ground_truth[i], (3,4))

In [28]:
def readPointCloud(file):
    pcd = np.fromfile(file, dtype=np.float32,count=-1)
    return pcd.reshape([-1,4])

In [15]:
#Visualizing first frame (LiDAR data from first bin file)
pcd1 = o3d.geometry.PointCloud()
points = readPointCloud("data/LiDAR/000010.bin")
pcd1.points = o3d.utility.Vector3dVector(points[:, :3]) #Taking x,y,z coordinates
o3d.visualization.draw_geometries([pcd1])
o3d.io.write_point_cloud("results/2/pcd1.ply",pcd1)

True

# Transform 

The point cloud obtained is with respect to the LiDAR frame. The poses however, are in the camera frame. If we want to combine the point clouds from various frames, we need to bring them to the camera frame. 

1. Refer to the image below and apply the required transformation to the point cloud. 

2. Then, register all point clouds into a common reference frame and visualise it (Open3D). It is helpful to use homogeneous coordinates to keep track of the different frames.

3. Write a function to transform the registered point cloud from the world to the $i^{th}$ camera frame, wherein $i$ is the input to the function.

4. \[Bonus\] Move around in the registered point cloud using arrow keys like you would do in a game. For this you will have to regularly transform the entire registered world to your current camera frame and visualize repeatedly. You may choose to avoid visualizing points that are behind the camera in this case as they are not visible from the scene. You may also visualize points at a max depth to make the process easier.

![](./img/transform.png)

In [21]:
# Function to transform from ith camera frame to 0th camera frame
# Vector [0 0 0 1] is appended to end of each pose matrix to make it a 4x4 matrix
def cam_i_to_cam_0(i):
    T_mat = np.reshape(ground_truth[i][:], (3,4))
    extra_row = np.array([0 ,0 , 0, 1])
    T_mat = np.vstack((T_mat, extra_row))
    return T_mat

In [26]:
# Registering all point clouds to a common world frame and visualizing
for i in range(10,87):
    pcd = readPointCloud("data/LiDAR/" + str(i).zfill(6) + ".bin")
    shape = pcd.shape[0]
    lidar_to_cami = np.array([[0,-1,0,0],[0,0,-1,0],[1,0,0,0],[0,0,0,1]])
    ones = np.ones((shape,1))
    pcd = np.hstack((pcd[:,:3],ones)) #Converting to homogenous coordinates by adding 4th coordinate of 1
    pcd_points = np.dot(np.asarray(pcd),lidar_to_cami.T) #LiDAR to ith camera frame
    i_to_0_mat = cam_i_to_cam_0(i - 10) #Taking first 77 rows
    pcd_points = np.dot(pcd_points, i_to_0_mat.T) #ith camera frame to 0th camera frame
    cam0_to_world = np.array([[0,0,1,0],[-1,0,0,0],[0,-1,0,0],[0,0,0,1]])
    pcd_points = np.dot(pcd_points, cam0_to_world.T) #0th camera frame to world frame

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pcd_points[:,:3])
    if i==10:
        pcd_final = o3d.geometry.PointCloud()
        pcd_final += pcd
    else:
        pcd_final += pcd
final_pcd = pcd_final.voxel_down_sample(voxel_size = 0.5)
o3d.visualization.draw_geometries([final_pcd])
o3d.io.write_point_cloud("results/3/final_pcd.ply", final_pcd)

True

In [27]:
#Function to transform registered point cloud from world to i'th camera frame
def world_to_cam_i(i):
    final_pcd = np.asarray(o3d.io.read_point_cloud("results/3/final_pcd.ply").points).reshape([-1, 4])
    world_to_cam0 = np.array([[0,-1,0,0],[0,0,-1,0],[1,0,0,0],[0,0,0,1]])
    shape = final_pcd.shape[0]
    ones = np.ones((shape,1))
    final_pcd = np.hstack((final_pcd[:,:3],ones)) #Converting to homogenous coordinates by adding 4th coordinate of 1
    final_pcd_points = np.asarray(final_pcd)
    final_pcd_points = np.dot(final_pcd_points, world_to_cam0.T) #World to 0th camera frame
    zero_to_i_mat = np.linalg.inv(cam_i_to_cam_0(i))
    final_pcd_points = np.dot(final_pcd_points, zero_to_i_mat.T)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(final_pcd_points[:, :3])
    return pcd

# Occupancy Map

Occupancy grid maps are discrete fine grain grid maps. These maps can be either 2-D or 3-D. Each cell in the occupancy grid map contains information on the physical objects present in the corresponding space. Since these maps shed light on what parts of the environment are occupied, and what is not, they are really useful for path planning and navigation.

Occupancy grid maps are probabilistic in nature due to noisy measurements. Each cell can have three states: Occupied, unoccupied, and unknown. For the purpose of this assignment, you can ignore the unknown and work in a binary setting where 1 is occupied and 0 is unoccupied.

1. The task here is to create an occupancy map for each LiDAR scan. You do not need to apply bayesian update rules here, just keep it simple. 

2. Now, using the *registered* point cloud, generate occupancy maps for each frame. What difference do you expect to see between the two methods?

You can mark a cell as occupied based on a threshold of how many different z values are there for a particular (x,y) cell.

In [6]:
#Function to compute map for a given point cloud
def computeMap(pcd):
    pcd_rounded = np.round(np.asarray(pcd.points))
    pcd_table = pd.DataFrame(data = pcd_rounded,columns=['x','y','z'])
    pcd_table.drop_duplicates(subset = ['y','x','z'],inplace=True)
    count = pcd_table.groupby(['x','y']).count()
    point = pcd_table.groupby(['x','y']).count().index
    pcd_table_2 = pd.DataFrame({'Point':point.values,'Count':count['z']})
    pcd_table_2.reset_index(drop=True,inplace=True)
    points_array = pcd_table_2['Point'].values
    img = np.zeros((400,400,1), np.uint8)
    l = len(points_array)
    for j in range(l):
        ind = (points_array[j][0] + 200,points_array[j][1]+ 200)
        count = pcd_table_2.iloc[j,1]
        if (count>2):
            img[int(ind[0])][int(ind[1])] = 1
    return img

In [28]:
#Creating occupancy maps for each LiDAR scan
for i in range(10,87):
    pcd = readPointCloud("data/LiDAR/" + str(i).zfill(6) + ".bin")
    shape = pcd.shape[0]
    lidar_to_cami = np.array([[0,-1,0,0],[0,0,-1,0],[1,0,0,0],[0,0,0,1]])
    ones = np.ones((shape,1))
    pcd = np.hstack((pcd[:,:3],ones)) #Converting to homogenous coordinates by adding 4th coordinate of 1
    pcd_points = np.dot(np.asarray(pcd),lidar_to_cami.T) #LiDAR to ith camera frame
    i_to_0_mat = cam_i_to_cam_0(i - 10)
    pcd_points = np.dot(pcd_points, i_to_0_mat.T) #ith camera frame to 0th camera frame
    cam0_to_world = np.array([[0,0,1,0],[-1,0,0,0],[0,-1,0,0],[0,0,0,1]])
    pcd_points = np.dot(pcd_points, cam0_to_world.T) #0th camera frame to world frame

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pcd_points[:,:3])
   
    img = computeMap(pcd)
    img = cv.resize(img,(700,700))
    filename = 'results/4/4.1/map'+str(i)+'.png'
    im = cv.imwrite(filename,img.T*255)

In [7]:
#Creating occupancy map for registered point cloud
for i in range(10,87):
    pcd = readPointCloud("data/LiDAR/" + str(i).zfill(6) + ".bin")
    shape = pcd.shape[0]
    lidar_to_cami = np.array([[0,-1,0,0],[0,0,-1,0],[1,0,0,0],[0,0,0,1]])
    ones = np.ones((shape,1))
    pcd = np.hstack((pcd[:,:3],ones)) #Converting to homogenous coordinates by adding 4th coordinate of 1
    pcd_points = np.dot(np.asarray(pcd),lidar_to_cami.T) #LiDAR to ith camera frame
    i_to_0_mat = cam_i_to_cam_0(i - 10)
    pcd_points = np.dot(pcd_points, i_to_0_mat.T) #ith camera frame to 0th camera frame
    cam0_to_world = np.array([[0,0,1,0],[-1,0,0,0],[0,-1,0,0],[0,0,0,1]])
    pcd_points = np.dot(pcd_points, cam0_to_world.T) #0th camera frame to world frame

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pcd_points[:,:3])
    if i==10:
        pcd_final = o3d.geometry.PointCloud()
        pcd_final += pcd
    else:
        pcd_final += pcd

img = computeMap(pcd_final)
img = cv.resize(img,(700,700))
filename = 'results/4/4.2/final_map.png'
im = cv.imwrite(filename,img.T*255)

## Difference in results

In the first case, when occupancy maps are created for single LiDAR scans, the maps appear sparse in terms of white patches. But in the second case when an occupancy map is created for a single registered point cloud, all the LiDAR scans are taken together, and the resultant map is denser since the white patches are more distinct in comparison.